In [65]:
# Inital state probability matrix, given
PI = [0.5,0.5]
# PI = {'Healthy': 0.5, 'Depressed': 0.5}
# state transition probability matrix, given
A = [[0.8,0.2],[0.001,0.999]]
# A = {
#     'Healthy' : {'Healthy': 0.8, 'Depressed': 0.2},
#     'Depressed' : {'Healthy': 0.001, 'Depressed': 0.999},
# }
# Emission or observation probability matrix, given
B = [[0.5,0.1,0.3,0.1,0.0],[0.05,0.35,0.2,0.2,0.2]]
# B = {
#     'Healthy' : {'Movement': 0.5, 'Passive': 0.1, 'Active' : 0.3, 'Texting' : 0.1, 'Access' : 0.0},
#     'Depressed' : {'Movement': 0.05, 'Passive': 0.35, 'Active' : 0.2, 'Texting' : 0.2, 'Access' : 0.2},
# }

# state codes of the observed states, dummy
# Observables = [0,1,2,3,4]
# Observables = ["Movement","Passive","Active","Texting","Access"]
#Observables = ["Movement", "Texting", "Movement", "Texting"]
#states = ["Healthy","Depressed"]
states = [0, 1]
Observables = [0,3,0,3]
#Observables = [0,0,2]
#Observables = {"Movement":0, "Passive Social":1, "Active Social":2, "Texting":3, "Access Psych Site":4}

# Filtering

In [66]:
import numpy as np 
from numpy import matrix

def forward_1(PI, A, B, Observables):
    #rows = [0] * len(Observables)
    alpha_matrix = []
    curr = []
    for t,t_value in enumerate(Observables):
        nex = [0] * len(A)
        for a,a_value in enumerate(A): 
            if t == 0:
                nex[a] = PI[a] * B[a][t_value]
            else:
                for i in range(len(curr)):
                    nex[a] = sum(curr[i]*A[i][l] for l in range(len(A))) * B[a][t_value]
            
        alpha_matrix.append(nex)
        curr = nex
        
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [67]:
import numpy as np 
from numpy import matrix

def forward(PI, A, B, Observables):
    alpha_matrix = []
    curr = [0] *len(A)
    #nex = [0] * len(A)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
    alpha_matrix.append(curr)
    for t in range (1, len(Observables)):
        nex = [0] * len(A)
        for i in range (0, len(A)):
            for j in range (0, len(A)):
                #nex = alpha_matrix[t-1][j]
                t_value = Observables[t]
                nex[i] =  nex[i] + curr[j] * A[j][i]
                nex[i] = nex[i] * B[i][t_value] 
        alpha_matrix.append(nex)
        curr = nex
            
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix

In [68]:
def forward_normalized(PI, A, B, Observables):
    alpha = forward(PI, A, B, Observables)
    c = [0] * len(Observables)
    for i,i_value in enumerate(alpha):
        for j,j_value in enumerate(i_value):
            c[j] += i_value[j]
        c[j] = 1/c[j]
            
    
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha[i][j] = alpha[i][j] * c[j]
    
    return alpha, c

In [69]:
def forward_normalized_1(PI, A, B, Observables):
    alpha_matrix = []
    curr = [0] *len(A)
    c = [0] * len(Observables)
    for i in range (0, len(A)):
        curr[i] = PI[i] * B[i][0]
        c[0] = c[0] + curr[i]
    alpha_matrix.append(curr)
    
    c[0] = 1/c[0]
    for i in range (0, len(A)):
        alpha_matrix[0][i] = c[0] * alpha_matrix[0][i]
    
    for t in range (1, len(Observables)):
        nex = [0] * len(A)
        c[t] = 0
        for i in range (0, len(A)):
            for j in range (0, len(A)):
                #nex = alpha_matrix[t-1][j]
                t_value = Observables[t]
                nex[i] =  nex[i] + curr[j] * A[j][i]
                nex[i] = nex[i] * B[i][t_value] 
            c[t] = c[t] + nex[i]
            
        c[t] = 1/c[t]
        for i in range (0,len(A)):
            nex[i] = c[t] * nex[i]
        alpha_matrix.append(nex)
        curr = nex
        
        
            
    #alpha_matrix = np.transpose(alpha_matrix)

    return alpha_matrix,c

In [70]:
alpha = forward(PI, A, B, Observables)

In [71]:
alpha

[[0.25, 0.025],
 [0.0020025000000000004, 0.006995000000000001],
 [0.0004039975000000001, 0.0003504015000000001],
 [3.2670201500000015e-06, 7.324219970000003e-05]]

In [72]:
alpha_unorm = forward_1(PI, A, B, Observables)

In [73]:
alpha_unorm

[[0.25, 0.025],
 [0.0025000000000000005, 0.005000000000000001],
 [0.0025000000000000005, 0.00025000000000000006],
 [2.5000000000000008e-05, 5.0000000000000016e-05]]

# Evaluation

In [74]:
def evaluation_unnormalized(alpha):
    probability = 0
    for i in range(len(alpha)):
        for j in range(1, len(alpha[i])):
             probability += alpha[i][j]        
    return probability

In [75]:
evaluation_unnormalized(alpha_unorm)

0.030300000000000004

In [76]:
def evaluation_normalized(alpha, c):
    alpha_norm = alpha.copy()
    for i in range(len(alpha)):
        for j in range(len(alpha[i])):
            alpha_norm[i][j] = alpha[i][j] * c[j]
    
    probability = 0
    for i in range(len(alpha)):
        for j in range(1, len(alpha[i])):
             probability += alpha[i][j]
    return probability

In [77]:
alpha_norm,c = forward_normalized(PI, A, B, Observables)

In [78]:
alpha_norm

[[0.0631024411300375, 0.0006336495834284839],
 [0.0005054505534516005, 0.00017729515344328982],
 [0.00010197291384172933, 8.881270580308637e-06],
 [8.246277867440855e-07, 1.8563955731716337e-06]]

In [79]:
c

[0.25240976452015, 0.025345983337139354, 0, 0]

In [80]:
alpha_norm_1, c_1 = forward_normalized_1(PI, A, B, Observables)

In [81]:
alpha_norm_1

[[0.9090909090909091, 0.09090909090909091],
 [0.22256182272853567, 0.7774381772714642],
 [0.535522316439974, 0.46447768356002594],
 [0.04270099938811492, 0.957299000611885]]

In [82]:
c_1

[3.6363636363636362, 30.564045568213384, 11.926712522153396, 9.860236471879276]

In [83]:
evaluation_normalized(alpha_norm,c)

2.0826348495498708e-05

# Backward Probability

In [84]:
def backward(PI, A, B, Observables):
    
    beta = []
    b_prev = {}
    for i, value in enumerate(reversed(Observables)):
        b_curr = {}
        for state in states:
            if i == 0:
                b_curr[state] = 1 
            else:
                b_curr[state] = sum(A[state][l] * B[l][value] * b_prev[l] for l in states)

        beta.insert(0,list(b_curr.values()))
        b_prev = b_curr

    #p_beta = sum(PI[l] * B[l][Observables[0]] * b_curr[l] for l in states)
    
    return beta #no of initial states * length of observation sequence

In [85]:
beta = backward(PI, A, B, Observables)

In [86]:
def backward_normalized(PI, A, B, Observables):
    alpha,c = forward_normalized(PI, A, B, Observables)
    beta = backward(PI, A, B, Observables)


    for i in range(len(beta)):
        for j in range(len(beta[i])):
            beta[i][j] = beta[i][j] * (1/c[j])
            
    return beta

In [87]:
backward_normalized(PI, A, B, Observables)

[[0.05557791762402325, 0.02063239952240188],
 [0.13794236552691078, 0.3993102128008534],
 [1.6243428647835434, 1.9904534509054082],
 [3.961811865325715, 39.453983169581925]]

# Smoothing by Forward-Backward Algorithm

In [88]:
def smoothed_probability(alpha, beta, noOfStates, noOfTimeSteps):
    matrix = []
    for t in range(len(alpha)):
        curr = [0] *len(A)
        for i in range(len(alpha[t])):
            sum_curr = 0
            for j in range(len(beta[t])):
                sum_curr += alpha[t][j] * beta[t][j]
            curr[i] = (alpha[t][i] * beta[t][i]) / sum_curr
        matrix.append(curr)  
                
    return matrix

In [89]:
smoothed_probability(alpha,beta,len(alpha), len(beta))

[[0.9962860631362895, 0.003713936863710455],
 [0.49618300053009035, 0.5038169994699098],
 [0.9035671451814147, 0.09643285481858543],
 [0.042700999388114926, 0.957299000611885]]

In [90]:
alpha*beta

TypeError: can't multiply sequence by non-int of type 'list'

In [91]:
np.matmul(alpha*beta)

TypeError: can't multiply sequence by non-int of type 'list'